# Unit Tests Demo

This notebook demonstrates key unit tests for individual analysis modules.

## Test Categories

1. **phi2ne Module**: Phase-to-density conversion, CDM, IQ, FPGA methods
2. **Phase Detector**: STFT ridge analysis, phase extraction
3. **Spectrum Analysis**: STFT, CWT, FFT frequency detection
4. **CORDIC**: Rotation and vectoring modes

## Usage

Run each cell sequentially to explore the test functionality.


In [ ]:
# Setup
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Add project root to path
project_root = Path().resolve().parent.parent.parent
sys.path.insert(0, str(project_root))

from ifi.analysis.phi2ne import PhaseConverter, _normalize_iq_signals, _calculate_differential_phase
from ifi.analysis.phase_analysis import STFTRidgeAnalyzer
from ifi.analysis.spectrum import SpectrumAnalysis
from ifi.analysis.phase_analysis import CORDICProcessor

print("Setup complete!")


## 1. phi2ne Module Tests

### Test: IQ Signal Normalization


In [ ]:
# Test IQ signal normalization
i_signal = np.array([1.0, 2.0, 3.0, 4.0])
q_signal = np.array([0.0, 1.0, 2.0, 3.0])

i_norm, q_norm = _normalize_iq_signals(i_signal, q_signal)

# Check normalization: magnitude should be approximately 1
magnitudes = np.sqrt(i_norm**2 + q_norm**2)
print(f"Magnitudes: {magnitudes}")
print(f"All close to 1.0: {np.allclose(magnitudes, 1.0, rtol=1e-10)}")
print(f"Normalized I: {i_norm}")
print(f"Normalized Q: {q_norm}")


### Test: Phase Calculation (CDM Method)


In [ ]:
# Test CDM phase calculation
fs = 50e6
f0 = 8e6
duration = 0.010
phase_offset = 0.75 * np.pi

# Generate test signals
t = np.arange(0, duration, 1/fs)
ref = np.sin(2 * np.pi * f0 * t)
probe = np.sin(2 * np.pi * f0 * t + phase_offset)

# Calculate phase using CDM
phase_converter = PhaseConverter()
phase = phase_converter.calc_phase_cdm(ref, probe, fs, f0, isbpf=False, islpf=True, isconj=False, iszif=False)

# Unwrap and check average offset
phase_u = np.unwrap(phase)
mean_offset = (phase_u.mean() + np.pi) % (2 * np.pi) - np.pi

print(f"Expected phase offset: {phase_offset:.4f} rad")
print(f"Detected mean offset: {mean_offset:.4f} rad")
print(f"Error: {abs(mean_offset - phase_offset):.4f} rad")
print(f"Test passed: {abs(mean_offset - phase_offset) < 0.05}")
